In [1]:
import pandas as pd

# CSV 파일 로드
file_path = './ChatbotData.csv'
df = pd.read_csv(file_path) #
questions = df['Q'].tolist()
answers = df['A'].tolist()


In [2]:
with open("training_texts.txt", "w", encoding="utf-8") as f:
    for line in questions + answers:
        f.write(line.strip() + "\n")

In [3]:
import sentencepiece as spm

# SentencePiece 학습
spm.SentencePieceTrainer.train(
    input="training_texts.txt",  # 학습 데이터 파일
    model_prefix="chatbot_spm",  # 생성될 모델 이름 (chatbot_spm.model, chatbot_spm.vocab)
    vocab_size=8000,            # 서브워드 사전 크기
    model_type="bpe"            # 모델 유형 (bpe, unigram, char 등 선택 가능)
)

In [4]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("chatbot_spm.model")

True

In [5]:
#데이터 전처리 인코딩
# 질문과 답변을 토크나이즈
encoded_questions = [sp.encode(q, out_type=int) for q in questions]
encoded_answers = [sp.encode(a, out_type=int) for a in answers]

In [6]:
#패딩 처리
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 19
padded_questions = pad_sequences(encoded_questions, maxlen=max_length, padding="post")
padded_answers = pad_sequences(encoded_answers, maxlen=max_length, padding="post")

In [7]:
#훈련 및 테스트 데이터 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_questions, padded_answers, test_size=0.2, random_state=42
)

In [8]:
#타겟 데이터 준비
decoder_input_data = y_train[:, :-1]  # 끝에서 한 단계 잘라냄
decoder_output_data = y_train[:, 1:]  # 시작에서 한 단계 잘라냄

In [9]:
#데이터 전처리
max_length = 19
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_length - 1, padding='post')  # 마지막 길이 조정
decoder_output_data = pad_sequences(decoder_output_data, maxlen=max_length - 1, padding='post')  # 타겟 길이 맞춤

In [10]:
print("X_train shape:", X_train.shape)
print("decoder_input_data shape:", decoder_input_data.shape)
print("decoder_output_data shape:", decoder_output_data.shape)


X_train shape: (9458, 19)
decoder_input_data shape: (9458, 18)
decoder_output_data shape: (9458, 18)


In [11]:
#모델 정의
import tensorflow as tf
from tensorflow.keras import layers

vocab_size = 8000  # SentencePiece의 단어 수와 동일

# Encoder
encoder_input = layers.Input(shape=(max_length,))  # Encoder 입력 길이
encoder_embedding = layers.Embedding(vocab_size, 256)(encoder_input)
encoder_output, state_h, state_c = layers.LSTM(256, return_state=True)(encoder_embedding)

# Decoder
decoder_input = layers.Input(shape=(max_length - 1,))  # Decoder 입력 길이
decoder_embedding = layers.Embedding(vocab_size, 256)(decoder_input)
decoder_lstm = layers.LSTM(256, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = layers.Dense(vocab_size, activation="softmax")
decoder_output = decoder_dense(decoder_output)

# Seq2Seq 모델
model = tf.keras.Model([encoder_input, decoder_input], decoder_output)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

print("Model output shape:", model.output_shape)

Model output shape: (None, 18, 8000)


In [12]:
# 모델 학습
history = model.fit(
    [X_train, decoder_input_data],
    decoder_output_data,
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

Epoch 1/50


c:\Users\envym\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_5']. Received: the structure of inputs=('*', '*')
  warnings.warn(


119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 364ms/step - accuracy: 0.7125 - loss: 4.0141 - val_accuracy: 0.7866 - val_loss: 1.7582
Epoch 2/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 43s 363ms/step - accuracy: 0.7858 - loss: 1.7065 - val_accuracy: 0.7880 - val_loss: 1.6798
Epoch 3/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 383ms/step - accuracy: 0.7877 - loss: 1.6188 - val_accuracy: 0.7883 - val_loss: 1.6369
Epoch 4/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 49s 413ms/step - accuracy: 0.7913 - loss: 1.5418 - val_accuracy: 0.7897 - val_loss: 1.6112
Epoch 5/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 45s 380ms/step - accuracy: 0.7914 - loss: 1.5196 - val_accuracy: 0.7933 - val_loss: 1.5937
Epoch 6/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 45s 381ms/step - accuracy: 0.7954 - loss: 1.4819 - val_accuracy: 0.7956 - val_loss: 1.5766
Epoch 7/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 384ms/step - accuracy: 0.7996 - loss: 1.4330 - val_accuracy: 0.7976 - val_loss: 1.5601
Epoch 8/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 389ms/step - accuracy: 0.8003 - loss: 1.4093 - val

In [13]:
#모델 평가
# decoder_input_test와 decoder_output_test 준비
decoder_input_test = y_test[:, :-1]  # 마지막 토큰 제거
decoder_output_test = y_test[:, 1:]  # 첫 번째 토큰 제거

# 모델 평가
loss, accuracy = model.evaluate(
    [X_test, decoder_input_test],
    decoder_output_test
)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.8580 - loss: 1.3954
Test Loss: 1.433877944946289
Test Accuracy: 0.8556025624275208


In [14]:
#테스트 데이터 예측
# 테스트 데이터 샘플
sample_input = X_test[:1]  # 테스트 데이터의 첫 번째 문장
sample_decoder_input = decoder_input_test[:1]  # 대응하는 decoder_input

# 예측 수행
predicted_output = model.predict([sample_input, sample_decoder_input])

# 예측된 시퀀스 변환 (가장 높은 확률의 토큰 선택)
predicted_sequence = predicted_output.argmax(axis=-1)

# SentencePiece 디코더로 디코딩
predicted_text = sp.decode(predicted_sequence[0].tolist())
print("Predicted text:", predicted_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Predicted text: 사람이네요 하지 마세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 


In [15]:
#예측 결과와 타겟 비교
# 타겟 텍스트 디코딩
target_text = sp.decode(y_test[0].tolist())  # 실제 타겟 시퀀스 디코딩

print(f"Input: {sp.decode(X_test[0].tolist())}")
print(f"Target: {target_text}")
print(f"Predicted: {predicted_text}")

Input: 죽을거 같네 ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Target: 나쁜 생각 하지 마세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Predicted: 사람이네요 하지 마세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 


In [16]:
#BLUE 스코어 계산
from nltk.translate.bleu_score import sentence_bleu

# BLEU 스코어 계산
reference = [target_text.split()]  # 실제 타겟 텍스트
candidate = predicted_text.split()  # 예측된 텍스트
bleu_score = sentence_bleu(reference, candidate)

print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.8817122476287481


In [17]:
#예측 결과 시각화
# 테스트 데이터에서 샘플 몇 개 출력
num_samples = 5
for i in range(num_samples):
    input_text = sp.decode(X_test[i].tolist())  # 입력 문장
    target_text = sp.decode(y_test[i].tolist())  # 실제 타겟 문장
    predicted_sequence = model.predict([X_test[i:i+1], decoder_input_test[i:i+1]]).argmax(axis=-1)
    predicted_text = sp.decode(predicted_sequence[0].tolist())  # 예측된 문장

    print(f"Input: {input_text}")
    print(f"Target: {target_text}")
    print(f"Predicted: {predicted_text}")
    print("-" * 30)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Input: 죽을거 같네 ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Target: 나쁜 생각 하지 마세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Predicted: 사람이네요 하지 마세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Input: 내일 시험이야 ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Target: 컨디션 조절 하세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Predicted: 조절은. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Input: 정말.내 자신이 싫다 ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Target: 자신은 사랑해주세요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Predicted: 원망. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Input: 이별후 네달째 ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Target: 바쁘게 살면서 잊어가요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
Predicted: 지냈 사세요가요. ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  